In [21]:
import pandas as pd
import numpy as np

from dateutil import relativedelta
from datetime import datetime, timedelta

In [22]:
df = pd.read_excel('data/train.xlsx', parse_dates=True, date_format='%Y-%m-%d')
df.head()

,PRICES,Hour 01,Hour 02,Hour 03,Hour 04,Hour 05,Hour 06,Hour 07,Hour 08,Hour 09,...,Hour 15,Hour 16,Hour 17,Hour 18,Hour 19,Hour 20,Hour 21,Hour 22,Hour 23,Hour 24
0,2007-01-01,24.31,24.31,21.71,8.42,0.01,0.01,0.02,0.01,0.01,...,15.00,10.00,8.17,27.77,37.99,33.11,37.99,33.00,36.48,30.65
1,2007-01-02,16.01,11.00,9.01,7.50,9.00,7.45,16.50,28.01,29.96,...,37.57,36.45,37.99,53.00,59.69,50.09,50.00,36.22,31.09,29.84
2,2007-01-03,28.00,24.50,24.15,18.99,15.50,23.11,29.01,39.73,43.81,...,43.21,43.99,48.00,57.00,60.99,55.51,51.77,34.51,39.31,38.05
3,2007-01-04,31.01,28.69,27.00,24.20,12.02,27.00,28.00,34.85,41.31,...,40.01,38.00,42.66,53.00,58.99,53.50,44.01,35.00,34.46,31.57
4,2007-01-05,22.00,19.50,17.55,15.91,11.40,22.65,27.30,34.00,40.01,...,40.06,38.00,38.61,49.84,53.00,45.91,36.74,29.61,30.61,27.51


In [23]:
data_cols = df.columns[1:-1]

In [24]:

df['PRICES_str'] = df.PRICES.dt.strftime('%Y-%m-%d')

df[['Year', 'Month', 'Day']] = df['PRICES_str'].str.split('-', expand=True)
df.Month = pd.to_numeric(df.Month)
df['Delta'] = abs(df.Month.values-7)
df.drop(['Year', 'Month', 'Day', 'PRICES_str'], axis='columns',inplace=True)
df.head()

,PRICES,Hour 01,Hour 02,Hour 03,Hour 04,Hour 05,Hour 06,Hour 07,Hour 08,Hour 09,...,Hour 16,Hour 17,Hour 18,Hour 19,Hour 20,Hour 21,Hour 22,Hour 23,Hour 24,Delta
0,2007-01-01,24.31,24.31,21.71,8.42,0.01,0.01,0.02,0.01,0.01,...,10.00,8.17,27.77,37.99,33.11,37.99,33.00,36.48,30.65,6
1,2007-01-02,16.01,11.00,9.01,7.50,9.00,7.45,16.50,28.01,29.96,...,36.45,37.99,53.00,59.69,50.09,50.00,36.22,31.09,29.84,6
2,2007-01-03,28.00,24.50,24.15,18.99,15.50,23.11,29.01,39.73,43.81,...,43.99,48.00,57.00,60.99,55.51,51.77,34.51,39.31,38.05,6
3,2007-01-04,31.01,28.69,27.00,24.20,12.02,27.00,28.00,34.85,41.31,...,38.00,42.66,53.00,58.99,53.50,44.01,35.00,34.46,31.57,6
4,2007-01-05,22.00,19.50,17.55,15.91,11.40,22.65,27.30,34.00,40.01,...,38.00,38.61,49.84,53.00,45.91,36.74,29.61,30.61,27.51,6


In [29]:
data = pd.DataFrame(columns=['Date', 'Price', 'Summer_delta', 'Weekday'])
for i, row in df.iterrows():
    day = row.PRICES
    for j, hour in enumerate(data_cols):
        hour_int = int(hour[-2:])
        time = day + timedelta(hours=hour_int)
        weekday = day.weekday()
        data.loc[i*24+j] = [time, row[hour], row.Delta, weekday]

data.head()

,Date,Price,Summer_delta,Weekday
0,2007-01-01 01:00:00,24.31,6,0
1,2007-01-01 02:00:00,24.31,6,0
2,2007-01-01 03:00:00,21.71,6,0
3,2007-01-01 04:00:00,8.42,6,0
4,2007-01-01 05:00:00,0.01,6,0


In [30]:
data.to_csv('data/train_preprocessed.csv')

In [31]:
val = pd.read_excel('data/validate.xlsx', parse_dates=True, date_format='%Y-%m-%d')

val['PRICES_str'] = val.PRICES.dt.strftime('%Y-%m-%d')
val[['Year', 'Month', 'Day']] = val['PRICES_str'].str.split('-', expand=True)
val.Month = pd.to_numeric(val.Month)
val['Delta'] = abs(val.Month.values-7)
val.drop(['Year', 'Month', 'Day', 'PRICES_str'], axis='columns',inplace=True)


data_val = pd.DataFrame(columns=['Date', 'Price', 'Summer_delta', 'Weekday'])
for i, row in val.iterrows():
    day = row.PRICES
    for j, hour in enumerate(data_cols):
        hour_int = int(hour[-2:])
        time = day + timedelta(hours=hour_int)
        data_val.loc[i*24+j] = [time, row[hour], row.Delta, day.weekday()]

data_val.to_csv('data/validate_preprocessed.csv')

In [32]:
data.head(20)

,Date,Price,Summer_delta,Weekday
0,2007-01-01 01:00:00,24.31,6,0
1,2007-01-01 02:00:00,24.31,6,0
2,2007-01-01 03:00:00,21.71,6,0
3,2007-01-01 04:00:00,8.42,6,0
4,2007-01-01 05:00:00,0.01,6,0
5,2007-01-01 06:00:00,0.01,6,0
6,2007-01-01 07:00:00,0.02,6,0
7,2007-01-01 08:00:00,0.01,6,0
8,2007-01-01 09:00:00,0.01,6,0
9,2007-01-01 10:00:00,6.31,6,0
